In [1]:
!pip install nltk
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np

import string
from string import digits

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import TweetTokenizer

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
train = pd.read_csv("SRC_projects/nlp_disaster/train.csv")
test = pd.read_csv("SRC_projects/nlp_disaster/test.csv")
sample_sub = pd.read_csv("SRC_projects/nlp_disaster/sample_submission.csv")

In [3]:
print("nombre de lignes pour le train set: ",train.shape[0])
display(train.head(2))
print("nombre de lignes pour le test set: ",test.shape[0])
display(test.head(2))

In [ ]:
print("Pourcentage de valeurs manquantes sur le train set:")
print(train.isna().sum()*100 / train.shape[0])
print("Pourcentage de valeurs manquantes sur le test set:")
print(test.isna().sum()*100 / test.shape[0]) 
#On peut déja enlever la colonne location
#Je vais faire un dropna pour enlever les valeurs manquantes, la colonne keyword pourra m'être utile

In [ ]:
#Opérations sur le train set:

train = train.drop('location', axis=1)
train = train.dropna()
train = train.reset_index()
train = train.drop('id',axis=1)
train = train.drop('index', axis=1)

In [ ]:
#Opérations sur le test set:

test = test.drop('location', axis=1)
test = test.dropna()
test = test.reset_index()
test = test.drop('index', axis=1)

In [ ]:
plt.pie(train['target'].value_counts(),
       labels=train['target'].unique(),
       autopct="%.1f",
        shadow=True,
        startangle=90)

In [ ]:
def preprocessing(df): #fonction qui va effectuer le nettoyage et l'encodage de nos datasets
    
    table = str.maketrans('', '', digits) #va me permettre d'enlever les numéros de la colonne text
    
    tt = TweetTokenizer() #fonction créee pour analyser les tweets, les convertir en petits tokens
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
    
    df['text_clean'] = df['text'].apply(lambda x: x.translate(table)).apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)).lower())
    # On supprime les ponctuations, les symboles et on enlève toutes les majuscules
    
    df['text_tokenized'] = df['text_clean'].apply(tt.tokenize)
    #On applique le tweet tokenizer sur la colonne de text clean pour obtenir de petits tokens
    tokenizer.fit_on_texts(df.text_tokenized)
    
    test['text_encoded'] = tokenizer.texts_to_sequences(test.text_tokenized) 
    #On crée une colonne encodée, chaque token est représenté par un chiffre ou nombre 
    
    df['len_text_encoded'] = df['text_encoded'].apply(lambda x: len(x))
    #On crée une colonne qui représente la longueur de chaque texte encodé
    
    df = df[df['len_text_encoded'] != 0] #J'applique un masque pour enlever tous les textes encodés nuls
    return df.head()